In [1]:
import pandas as pd
import numpy as np
import pickle
from selenium import webdriver
import requests
import time

In [2]:
with open('kickstarter_data.pickle', 'rb') as f:
    df = pickle.load(f)

In [3]:
print(df.shape)
df.head()

(26655, 19)


,backers_count,created_at,deadline,goal,launched_at,name,spotlight,staff_pick,state,usd_pledged,category,subcategory,project_url,project_life,campaign_length,launch_month,deadline_month,daily_amount_pledged,daily_backers
0,8,2015-10-21 00:29:19,2015-12-04 15:05:15,18000.0,2015-11-04 15:05:15,Sound Affections - old fashion Greeting Cards ...,True,False,successful,19120.0,technology,sound,https://www.kickstarter.com/projects/206159179...,44,30,2015-11,2015-12,637.0,0.0
1,246,2018-12-12 13:46:33,2019-01-14 01:30:02,100.0,2018-12-15 01:30:02,Dragon Scales: A Tabletop RPG Game of Chance™,True,False,successful,11940.0,games,tabletop games,https://www.kickstarter.com/projects/308858186...,32,30,2018-12,2019-01,398.0,8.0
2,457,2017-03-25 22:41:02,2017-05-28 20:59:00,10000.0,2017-04-28 18:00:23,Pokitto - easy-to-learn and program Gaming Gad...,True,True,successful,27235.0,technology,diy electronics,https://www.kickstarter.com/projects/175428417...,63,30,2017-04,2017-05,908.0,15.0
3,171,2018-09-07 19:37:50,2018-10-15 01:00:00,6000.0,2018-09-11 22:38:15,Kitten Stuff Done: Playfully Productive,True,False,successful,6973.0,games,playing cards,https://www.kickstarter.com/projects/victoragr...,37,33,2018-09,2018-10,211.0,5.0
4,62,2018-04-15 23:03:18,2018-09-02 14:01:16,3500.0,2018-07-04 14:01:16,The Hand Glider-A timeless toy for everyone!,True,False,successful,3560.0,design,product design,https://www.kickstarter.com/projects/109085546...,139,60,2018-07,2018-09,59.0,1.0


In [4]:
df.category.unique()

array(['technology', 'games', 'design'], dtype=object)

## Additional filtering to limit the dataset

In [5]:
# Only tech projects
tech_projects = df.loc[df['category'] == 'technology'].reset_index(drop=True)
tech_projects.shape

(12472, 19)

In [6]:
# Only completed projects (successful or failed)
tech_projects = tech_projects.loc[df['state'].isin(['successful', 'failed'])].reset_index(drop=True)
tech_projects.shape

(12472, 19)

In [7]:
# Looking only at hardware projects
tech_subcategories = tech_projects.loc[tech_projects['subcategory'].isin(['technology', 'wearables', 'hardware', 'gadgets'])].reset_index(drop=True)
tech_subcategories.shape

(4668, 19)

In [35]:
# Save dataframe to pickle file
with open('tech_projects.pickle', 'wb') as outfile:
    pickle.dump(tech_subcategories, outfile)

## Grab project URLs and scrape for social links

In [8]:
# Select project names and urls
unique_projects = tech_subcategories.loc[:, ['name', 'project_url']]
print(unique_projects.shape)
unique_projects

(4668, 2)


,name,project_url
0,Felik - Intelligent Pet Companion,https://www.kickstarter.com/projects/angrylych...
1,The Stuff Stopper,https://www.kickstarter.com/projects/211910141...
2,FreshFridge: The First Affordable Smart Fridge...,https://www.kickstarter.com/projects/freshfrid...
3,"Solamigo,The World 1st Thin Film Solar& Anti T...",https://www.kickstarter.com/projects/134491136...
4,303 SS Pen 2,https://www.kickstarter.com/projects/183914960...
...,...,...
4663,Bright Lock - High Tech Illuminated Doorknob,https://www.kickstarter.com/projects/illumimat...
4664,I-TapR2: The Internet of Beer Pouring,https://www.kickstarter.com/projects/itap/i-ta...
4665,The aF⁴ - Auto Frozen Fish Food Feeder,https://www.kickstarter.com/projects/af4/the-a...
4666,BravoWalk Dog Collar,https://www.kickstarter.com/projects/bravowalk...


In [9]:
# Store urls as an array
urls = unique_projects['project_url']

## THIS WORKS. REPEAT THIS LOOP, INCREMENTING THE COUNT EACH LOOP, APPENDING TO THE SAME LIST AND CREATE A DATAFRAME FROM IT ONCE IT'S ALL IN THERE.

In [10]:
## One URL at a time
def get_social_links(url):
    """
    Loop through a pandas Series of urls and scrape their web pages for links to their social accounts.

    :param row (Series): A series containing standard 'http://' web addresses.
    :return: A dataframe containing the project urls in one column and a list of all scraped socials in another column.
    """
    driver = webdriver.PhantomJS()

    data = []
    
    driver.get(url)

    try:
        # Find hrefs by xpath
        links = driver.find_elements_by_xpath("//a[@href]")

        # Loop through the links and add socials to a list
        socials = ['facebook', 'twitter', 'instagram', 'twitch', 'youtube']
        share_links = ['intent', 'share', 'kickstarter']
        social_links = []

        for elem in links:
            if any(social in elem.get_attribute('href') for social in socials):
                if any(share in elem.get_attribute('href') for share in share_links):
                    pass
                else:
                    social_links.append(elem.get_attribute('href'))
    except:
        social_links = []

    data.append({'project_url': url, 'social_links': social_links})

    return data

<font color='red'>DON'T RUN THIS YET YOU BETCH</font>

In [13]:
# DON'T RUN THIS RN
list_of_stuff = []

In [26]:
count = 4263
for i in range(count, len(urls)+1):
    list_of_stuff.extend(get_social_links(urls[i]))
    count += 1
    print(count, requests.get(urls[i]).status_code)
    
    if (i+1 % 150 == 0):
        time.sleep(200)

/Users/baka_brooks/anaconda3/envs/metis/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


4264 200


/Users/baka_brooks/anaconda3/envs/metis/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


4265 200


/Users/baka_brooks/anaconda3/envs/metis/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


...

/Users/baka_brooks/anaconda3/envs/metis/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


4667 200


/Users/baka_brooks/anaconda3/envs/metis/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


4668 200


KeyError: 4668

<font color='red'>Run these cells after done loading links into list</font>

In [29]:
social_df = pd.DataFrame(list_of_stuff)

In [32]:
# Save dataframe to pickle file
with open('social_data.pickle', 'wb') as outfile:
    pickle.dump(social_df, outfile)

## ARCHIVED --> This is the function to input the list of URLs rather than one at a time.

In [20]:
# List of URLS
def get_social_links(list_of_urls):
    """
    Loop through a pandas Series of urls and scrape their web pages for links to their social accounts.

    :param row (Series): A series containing standard 'http://' web addresses.
    :return: A dataframe containing the project urls in one column and a list of all scraped socials in another column.
    """
    driver = webdriver.PhantomJS()

    data = []
    count = 0
    for url in list_of_urls:
        driver.get(url)

        try:
            # Find hrefs by xpath
            links = driver.find_elements_by_xpath("//a[@href]")

            # Loop through the links and add socials to a list
            socials = ['facebook', 'twitter', 'instagram', 'twitch', 'youtube']
            share_links = ['intent', 'share', 'kickstarter']
            social_links = []

            for elem in links:
                if any(social in elem.get_attribute('href') for social in socials):
                    if any(share in elem.get_attribute('href') for share in share_links):
                        pass
                    else:
                        social_links.append(elem.get_attribute('href'))
        except:
            social_links = []
            
        count += 1
            
        print(count, requests.get(url).status_code)

        data.append({'project_url': url, 'social_links': social_links})

    return data